## Splitting off the scraper from the main project

In [1]:
import numpy as np
import glob
import re
from collections import defaultdict
from bs4 import BeautifulSoup
import urllib
from urllib.error import HTTPError
import time
import random
import re
import traceback


# Tor stuff, this might not all be needed
# TODO Clean it up later
import stem
import requests
from stem import Signal
from stem.control import Controller
from stem.util import term
import socks as pysocks
import socket

In [3]:
# This can only be done once
controller = Controller.from_port(port = 9051)

In [4]:
def new_ident():
    controller.authenticate()
    controller.signal(Signal.NEWNYM)
    pysocks.setdefaultproxy(pysocks.PROXY_TYPE_SOCKS5 , "127.0.0.1", 9050, True)
    socket.socket = pysocks.socksocket
    print('Connecting from {}'.format(requests.get('http://icanhazip.com').text))
    time.sleep(4)


for i in range(0,1):    
    new_ident()

Connecting from 109.163.234.4



In [7]:
# Drilling down through the directory structure with the scraper
def pull_lyrics():
    new_ident()
    
    u = urllib.request.urlopen("http://ohhla.com/all_five.html")
    soup = BeautifulSoup(u, 'html.parser')
    submitters = soup.select('#leftmain table pre a')
    sub_pages = []
    for link in submitters: 
        rel = (link.get('href'))
        if rel:
            sub_pages.append('http://ohhla.com/' + rel)
    print(len(sub_pages), 'sub pages found.')
    batch_step = 1 
    for i in range(49, len(sub_pages), batch_step):
        batch = sub_pages[i:i+batch_step]
        lyric_text(batch)
        print('{} OF {} COMPLETED'.format((1+i)*batch_step, len(sub_pages)))

# This didn't solve the issue, but will be useful once we get Tor working
# Break into a seperate function here to avoid time outs
def lyric_text(passed_pages):
    # Made passed pages a set to cancel out the duplication for artist pages
    # TODO, see if this works
    
    for artist in set(passed_pages):
        if artist:
            try:
                sub_u = urllib.request.urlopen(artist) 
                sub_soup = (BeautifulSoup(sub_u, 'html.parser'))
                lyrics_pages = sub_soup.select('body table tr a')
                for lyrics_page in lyrics_pages[5:]:
                    if lyrics_page: 
                        try:
#                             print('Artist:{}, Lyrics page:{}'.format(artist, lyrics_page))
                            ly_lnk = artist+lyrics_page.get('href')
                            if '#' not in ly_lnk: # Lyrics pages with # are just links to anchors on the same page
                                ly_u = urllib.request.urlopen(ly_lnk)
                                ly_soup = BeautifulSoup(ly_u, 'html.parser')
                                ly_txts = ly_soup.findAll(href=re.compile("\.txt$"))             
                                for ly_txt in ly_txts:

                                        attempt_lyrics_save(ly_txt, ly_lnk)
                            else:
                                print('Skipping {}, it has a #'.format(ly_lnk))

                        # Some of the artists are organized in an artist page, or we could need a new identy
                        # Artist page is more likely so we check for it first
                        except (TypeError, HTTPError) as e:
                            if '#' not in ly_lnk: # Lyrics pages with # are just links to anchors on the same page
                                try:
                                    # Switch identity in case it was part of the issue
                                    print('Using alternate workflow for artist page for: {}'.format(ly_lnk))
                                    ly_lnk = 'http://ohhla.com/'+lyrics_page.get('href')
                                    # Right now this is just checking that it does open
                                    ly_u = urllib.request.urlopen(ly_lnk)
                                    ly_soup = BeautifulSoup(ly_u, 'html.parser')
                                    ly_txt = re.search('[\w\.]*txt', ly_lnk).group(0)
                                    ly_lnk = ly_lnk[0:-len(ly_txt)]
#                                     print('Attempting lyrics save ly_txt:{}, ly_lnk:{}'.format(ly_txt, ly_lnk))
                                    attempt_lyrics_save(ly_txt, ly_lnk, direct_link=True)

                                except (TypeError, HTTPError) as e: 
                                    print('ly_lnk' + ' is not valid.  Skipping it.', e)      
                        # At this point we have a new identity, so check if that was the problem 
                        # This should only get hit if the format was normal *but* we exceeded the rate limit
                        except (TypeError, HTTPError):
                            print('Trying base format with new identity')
                            ly_lnk = artist+lyrics_page.get('href')
                            ly_u = urllib.request.urlopen(ly_lnk)
                            ly_soup = BeautifulSoup(ly_u, 'html.parser')
                            ly_txts = ly_soup.findAll(href=re.compile("\.txt$"))             
                            for ly_txt in ly_txts:

                                    attempt_lyrics_save(ly_txt, ly_lnk)

                        except Exception as e:
                            print('A non TypeError has occured. ', e)

            except Exception as e:
                print('Artist: {}\n raised {}'.format(artist, e))
                print(traceback.print_exc())
            time.sleep(1+(random.random()*2))


def attempt_lyrics_save(page_in, ly_lnk_in, final_run=False, direct_link=False):
    if direct_link is False:
        page = page_in.get('href')
    else:
        page = page_in
    lyrics = urllib.request.urlopen(ly_lnk_in + page)
    lyrics_soup = BeautifulSoup(lyrics, 'html.parser')
    lyrics_cln = lyrics_soup.select('body div pre')
    lyrics_cln = str(lyrics_cln)[7:-7] # Stripping out the pre tags (I know it's ugly)
    filename = page.rsplit('/', 1)[-1]    
    
    if len(lyrics_cln) > 0:
        with open('lyrics/' + filename, 'w+') as out:
            out.write(str(lyrics_cln))
        if final_run==True:
            print('After identity switching, the lyrics length is {}'.format(len(lyrics_cln)))

    elif final_run==True:
        # We're being denied now
        print('We are getting a blank lyrics page', ly_lnk_in+page) 
        new_ident()
        attempt_lyrics_save(page_in, ly_lnk_in, final_run=True)
pull_lyrics()



Connecting from 176.10.104.243

183 sub pages found.
50 OF 183 COMPLETED
51 OF 183 COMPLETED
52 OF 183 COMPLETED
53 OF 183 COMPLETED
54 OF 183 COMPLETED
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#eleven, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#ambition, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#folarin, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#gifted, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#festivus, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#albumabout, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#remix, it has a #
Skipping http://ohhla.com/YFA_wale.htmlYFA_wale.html#misc, it has a #
Using alternate workflow for artist page for: http://ohhla.com/YFA_wale.htmlhttp://www.datpiff.com/Wale-100-Miles-and-Running-mixtape.5625.html
ly_lnk is not valid.  Skipping it. HTTP Error 404: Not Found
Using alternate workflow for artist page for: http://ohhla.c

Traceback (most recent call last):
  File "<ipython-input-7-38ff115ab7ea>", line 29, in lyric_text
    sub_u = urllib.request.urlopen(artist)
  File "/Users/noah/anaconda3/envs/hhweb/lib/python3.6/urllib/request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/noah/anaconda3/envs/hhweb/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/Users/noah/anaconda3/envs/hhweb/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/Users/noah/anaconda3/envs/hhweb/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/Users/noah/anaconda3/envs/hhweb/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/Users/noah/anaconda3/envs/hhweb/lib/python3.6/urllib/request.py", line 650, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: H

156 OF 183 COMPLETED
157 OF 183 COMPLETED
158 OF 183 COMPLETED
159 OF 183 COMPLETED
160 OF 183 COMPLETED
161 OF 183 COMPLETED
162 OF 183 COMPLETED
163 OF 183 COMPLETED
164 OF 183 COMPLETED
165 OF 183 COMPLETED
166 OF 183 COMPLETED
167 OF 183 COMPLETED
168 OF 183 COMPLETED
169 OF 183 COMPLETED
170 OF 183 COMPLETED
171 OF 183 COMPLETED
172 OF 183 COMPLETED
173 OF 183 COMPLETED
174 OF 183 COMPLETED
175 OF 183 COMPLETED
176 OF 183 COMPLETED
177 OF 183 COMPLETED
178 OF 183 COMPLETED
179 OF 183 COMPLETED
180 OF 183 COMPLETED
181 OF 183 COMPLETED
182 OF 183 COMPLETED
183 OF 183 COMPLETED
